In [1]:
from openai import OpenAI
import json
import pandas as pd
import time
from tqdm import tqdm
from collections import defaultdict
import numpy as np
import random
import pickle
import re

In [2]:
client = OpenAI(api_key="sk-proj-3KhQdlgMCuoQc6EL2y1ZT3BlbkFJYHG9JgPGpOLsRnDSKuSs")
prompt_5_types = "asst_13QqvGJEmZcaizKEmeSZMQnC"

In [3]:
# prompt_3_types = "asst_zX008FyufXH0pgwjuUXAcZkX"

In [4]:
def predict_label(text):

    try:
        # Create a thread with a message.
        thread = client.beta.threads.create(
            messages=[
                {
                    "role": "user",
                    # Update this with the query you want to use.
                    "content": text,
                }
            ]
        )
    # Submit the thread to the assistant (as a new run).
        run = client.beta.threads.runs.create(thread_id=thread.id, assistant_id=prompt_5_types)

        # Wait for run to complete.
        while run.status != "completed":
            run = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)
            time.sleep(1)
        # else:
        #     print(f"🏁 Run Completed!")

        # Get the latest message from the thread.
        message_response = client.beta.threads.messages.list(thread_id=thread.id)
        messages = message_response.data

        # Print the latest message.
        latest_message = messages[0]
        return latest_message.content[0].text.value
    except:
        print(f"Run went wrong")
        return None

    

In [5]:
def extract_label_and_explanation(text):
    label_match = re.search(r"Label:(.*?)Explanation:", text, re.DOTALL)
    label_text = label_match.group(1).strip() if label_match else text

    # Extract text after "Explanation"
    explanation_match = re.search(r"Explanation:(.*)", text, re.DOTALL)
    explanation_text = explanation_match.group(1).strip() if explanation_match else text
    
    return label_text, explanation_text
    

Rerun experiment 3 times 

In [6]:
df_gold = pd.read_csv(r"C:\Users\Nursulu_1\Downloads\ContraDetect\automatic_stance_detection\data\qualtrics_survey\results\df_gold_all.csv")

In [7]:
statements = list(df_gold['Input'])

In [8]:
len(statements)

698

In [9]:
with open('new_run/answers_chatgpt4_turbo_5_types_run4.pkl', 'rb') as f:
    answers_chatgpt4_turbo_5_types_run4 = pickle.load(f)

In [10]:
# answers_chatgpt4_turbo_5_types_run4 = dict()
for sample in tqdm(statements):
    if sample not in answers_chatgpt4_turbo_5_types_run4:
        answers_chatgpt4_turbo_5_types_run4[sample] = dict()
        answer = predict_label(sample)
        try:
            label, explanation = extract_label_and_explanation(answer)
            answers_chatgpt4_turbo_5_types_run4[sample]['label'] = label
            answers_chatgpt4_turbo_5_types_run4[sample]['explanation'] = explanation
        except:
            print("Couldn't parse the answer for sample")
            print(sample)
            answers_chatgpt4_turbo_5_types_run4[sample]['label'] = answer
        with open('new_run/answers_chatgpt4_turbo_5_types_run4.pkl', 'wb') as f:
            pickle.dump(answers_chatgpt4_turbo_5_types_run4, f)
        

100%|██████████| 698/698 [27:56<00:00,  2.40s/it] 


In [8]:
answers_chatgpt35_turbo_5_types_run5 = dict()

In [7]:
with open('new_run/answers_chatgpt35_turbo_5_types_run2.pkl', 'rb') as f:
    answers_chatgpt35_turbo_5_types_run2 = pickle.load(f)

In [9]:
# answers_chatgpt35_turbo_5_types_run5 = dict()
for sample in tqdm(statements):
    answers_chatgpt35_turbo_5_types_run5[sample] = dict()
    answer = predict_label(sample)
    try:
        label, explanation = extract_label_and_explanation(answer)
        answers_chatgpt35_turbo_5_types_run5[sample]['label'] = label
        answers_chatgpt35_turbo_5_types_run5[sample]['explanation'] = explanation
    except:
        print("Couldn't parse the answer for sample")
        print(sample)
        answers_chatgpt35_turbo_5_types_run5[sample]['label'] = answer
    with open('new_run/answers_chatgpt35_turbo_5_types_run5.pkl', 'wb') as f:
        pickle.dump(answers_chatgpt35_turbo_5_types_run5, f)

100%|██████████| 698/698 [55:07<00:00,  4.74s/it] 


In [24]:
def predict_label_3_types(text):

    try:
        # Create a thread with a message.
        thread = client.beta.threads.create(
            messages=[
                {
                    "role": "user",
                    # Update this with the query you want to use.
                    "content": text,
                }
            ]
        )
    # Submit the thread to the assistant (as a new run).
        run = client.beta.threads.runs.create(thread_id=thread.id, assistant_id=prompt_3_types)

        # Wait for run to complete.
        while run.status != "completed":
            run = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)
            time.sleep(1)
        # else:
        #     print(f"🏁 Run Completed!")

        # Get the latest message from the thread.
        message_response = client.beta.threads.messages.list(thread_id=thread.id)
        messages = message_response.data

        # Print the latest message.
        latest_message = messages[0]
        return latest_message.content[0].text.value
    except:
        print(f"Run went wrong")
        return None

    

In [8]:
with open(r"C:\Users\Nursulu_1\Downloads\ContraDetect\automatic_stance_detection\notebooks\sample_to_responses_Part_1.pkl", 'rb') as f:
    sample_to_responses = pickle.load(f)

In [12]:
with open(r'C:\Users\Nursulu_1\Downloads\ContraDetect\automatic_stance_detection\notebooks\evaluating_models\answers_chatgpt_3_types.pkl', 'rb') as f:
    answers_chatgpt_3_types = pickle.load(f)
with open(r'C:\Users\Nursulu_1\Downloads\ContraDetect\automatic_stance_detection\notebooks\evaluating_models\answers_chatgpt_3_types_350_700.pkl', 'rb') as f:
    answers_chatgpt_3_types_pt2 = pickle.load(f)

In [13]:
### replace "\n" with "\n\n"
key_changes = dict()
for el in answers_chatgpt_3_types_pt2.keys():
    key_changes[el] = el.replace("\n", "\n\n")
    
# Rename keys
for old_key, new_key in key_changes.items():
    if old_key in answers_chatgpt_3_types_pt2:
        answers_chatgpt_3_types_pt2[new_key] = answers_chatgpt_3_types_pt2.pop(old_key)

In [14]:
annotated_samples = []
for el in answers_chatgpt_3_types:
    annotated_samples.append(el)
for el in answers_chatgpt_3_types_pt2:
    annotated_samples.append(el)

In [9]:
df = pd.read_csv(r"C:\Users\Nursulu_1\Downloads\ContraDetect\automatic_stance_detection\data\qualtrics_survey\all_samples_df.csv")

In [16]:
df_gold = pd.read_csv(r"C:\Users\Nursulu_1\Downloads\ContraDetect\automatic_stance_detection\data\qualtrics_survey\results\df_gold_all.csv")

In [19]:
statements = list(df_gold['Input'])

In [25]:
len(set(statements) - set(annotated_samples))

698

In [26]:
to_annotate = list(set(statements) - set(annotated_samples))

In [29]:
to_annotate[-1]

'Text 1: Europe needs tax cuts, not new taxes.\n\nText 2: We support the EU having the ability to levy its own taxes instead of relying on national contributions.'

In [30]:
answers_chatgpt4_3_types = dict()
for sample in tqdm(to_annotate):
    answers_chatgpt4_3_types[sample] = dict()
    answer = predict_label_3_types(sample)
    try:
        label, explanation = extract_label_and_explanation(answer)
        answers_chatgpt4_3_types[sample]['label'] = label
        answers_chatgpt4_3_types[sample]['explanation'] = explanation
    except:
        print("Couldn't parse the answer for sample")
        print(sample)
        answers_chatgpt4_3_types[sample]['label'] = answer
    with open('answers_chatgpt4_3_types_leftovers.pkl', 'wb') as f:
        pickle.dump(answers_chatgpt4_3_types, f)

100%|██████████| 102/102 [17:16<00:00, 10.16s/it]


In [13]:
answers_chatgpt35_turbo_3_types = dict()
for sample in tqdm(statements):
    answers_chatgpt35_turbo_3_types[sample] = dict()
    answer = predict_label_3_types(sample)
    try:
        label, explanation = extract_label_and_explanation(answer)
        answers_chatgpt35_turbo_3_types[sample]['label'] = label
        answers_chatgpt35_turbo_3_types[sample]['explanation'] = explanation
    except:
        print("Couldn't parse the answer for sample")
        print(sample)
        answers_chatgpt35_turbo_3_types[sample]['label'] = answer
    with open('answers_chatgpt35_turbo_3_types.pkl', 'wb') as f:
        pickle.dump(answers_chatgpt35_turbo_3_types, f)

100%|██████████| 700/700 [51:24<00:00,  4.41s/it] 


In [11]:
answers_chatgpt35_turbo_5_types = dict()
for sample in tqdm(statements):
    answers_chatgpt35_turbo_5_types[sample] = dict()
    answer = predict_label(sample)
    try:
        label, explanation = extract_label_and_explanation(answer)
        answers_chatgpt35_turbo_5_types[sample]['label'] = label
        answers_chatgpt35_turbo_5_types[sample]['explanation'] = explanation
    except:
        print("Couldn't parse the answer for sample")
        print(sample)
        answers_chatgpt35_turbo_5_types[sample]['label'] = answer
    with open('answers_chatgpt35_turbo_5_types_350_700.pkl', 'wb') as f:
        pickle.dump(answers_chatgpt35_turbo_5_types, f)
        

100%|██████████| 700/700 [49:19<00:00,  4.23s/it] 


TODO: run the second half

In [14]:
answers_chatgpt_3_types = dict()
for sample in tqdm(statements[350:]):
    answers_chatgpt_3_types[sample] = dict()
    answer = predict_label_3_types(sample)
    try:
        label, explanation = extract_label_and_explanation(answer)
        answers_chatgpt_3_types[sample]['label'] = label
        answers_chatgpt_3_types[sample]['explanation'] = explanation
    except:
        print("Couldn't parse the answer for sample")
        print(sample)
        answers_chatgpt_3_types[sample]['label'] = answer
    with open('answers_chatgpt_3_types_350_700.pkl', 'wb') as f:
        pickle.dump(answers_chatgpt_3_types, f)

  0%|          | 0/350 [00:00<?, ?it/s]